In [17]:
import pandas as pd
import yfinance as yf
from datetime import datetime
import pandas_datareader as pdr
import pandas_datareader.data as web

tickers = ['AAPL','NVDA','MSFT','LLY','DVA','COR','JPM','BAC','WFC',"AMZN", "NKE", "SBUX","GOOGL", "META", "VZ","BA", "MMM", "GE","PG", "KO", "PEP","XOM", "CVX", "SLB","NEE", "DUK", "SO","AMT", "SPG", "PLD","BHP", "RIO", "ECL"]
start_date = "2018-12-14"
end_date = "2024-03-15"

In [18]:
len(tickers)

33

# Data

5 Year Return

In [19]:
data = yf.download(tickers, start=start_date, end=end_date)
daily_returns = data['Adj Close'].pct_change()
#daily_returns.to_csv("Data/stock_5y_returns.csv")

[*********************100%***********************]  33 of 33 completed


fama french 3 factors

In [20]:
ff_factors = web.DataReader('F-F_Research_Data_Factors_daily', 'famafrench', start_date, end_date)[0]
#ff_factors.to_csv('Data/ff_factors 5y.csv')

50 days moving average

In [21]:
ma_50 = data['Adj Close'].rolling(window=50).mean()

suffix = '_ma50'
ma_50.columns = [f'{col}{suffix}' for col in ma_50.columns]

#ma_50.to_csv('Data/moving_average.csv')

14 days rsi

In [22]:
def calculate_rsi(data, window=14):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()

    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

rsi = calculate_rsi(data['Adj Close'], window=14)

suffix = '_rsi'
rsi.columns = [f'{col}{suffix}' for col in rsi.columns]

#rsi.to_csv('Data/rsi.csv')

macd

In [23]:
def calculate_macd(data, span1=12, span2=26, signal_span=9):
    ema_short = data.ewm(span=span1, adjust=False).mean()
    ema_long = data.ewm(span=span2, adjust=False).mean()
    macd = ema_short - ema_long
    signal = macd.ewm(span=signal_span, adjust=False).mean()
    return macd, signal

macd, signal = calculate_macd(data['Adj Close'])

suffix = '_macd'
macd.columns = [f'{col}{suffix}' for col in macd.columns]

suffix = '_macds'
signal.columns = [f'{col}{suffix}' for col in signal.columns]

#macd.to_csv('Data/macd.csv')
#signal.to_csv('Data/macd_signal.csv')

macros, foward filled, daily

In [24]:
indicators = {
    'GDP': 'GDP',
    'CPIAUCSL': 'CPI',  # Consumer Price Index for All Urban Consumers: All Items in U.S. City Average
    'UNRATE': 'Unemployment Rate',  # Unemployment Rate
    'FEDFUNDS': 'Federal Funds Rate',  # Effective Federal Funds Rate
    'RSXFS': 'Retail Sales'  # Retail Sales: Retail Trade and Food Services
}

macro = pdr.DataReader(list(indicators.keys()), 'fred', start_date, end_date)

macro.rename(columns=indicators, inplace=True)


#macro.to_csv('Data/macroeconomic_indicators.csv')

bollinger band

In [25]:
def calculate_bollinger_bands(data, window=20, num_of_std=2):
    rolling_mean = data.rolling(window=window).mean()
    rolling_std = data.rolling(window=window).std()

    upper_band = rolling_mean + (rolling_std * num_of_std)
    lower_band = rolling_mean - (rolling_std * num_of_std)

    return rolling_mean, upper_band, lower_band

middle_band, upper_band, lower_band = calculate_bollinger_bands(data['Adj Close'])

suffix = '_bm'
middle_band.columns = [f'{col}{suffix}' for col in middle_band.columns]

suffix = '_bu'
upper_band.columns = [f'{col}{suffix}' for col in upper_band.columns]

suffix = '_bl'
lower_band.columns = [f'{col}{suffix}' for col in lower_band.columns]

#middle_band.to_csv('Data/bollinger_middle.csv')
#upper_band.to_csv('Data/bollinger_upper.csv')
#lower_band.to_csv('Data/bollinger_lower.csv')

merge data

In [26]:
returns = pd.read_csv('Data/stock_5y_returns.csv')
factors = pd.read_csv('Data/ff_factors 5y.csv')
MA = pd.read_csv ("Data/moving_average.csv")
rsi = pd.read_csv('Data/rsi.csv')
macd = pd.read_csv('Data/macd.csv')

macd_s = pd.read_csv("Data/macd_signal.csv")

B_m = pd.read_csv('Data/bollinger_middle.csv')
B_u = pd.read_csv('Data/bollinger_upper.csv')
B_l = pd.read_csv('Data/bollinger_lower.csv')
macro = pd.read_csv('Data/macroeconomic_indicators.csv')
macro.columns = ['Date', 'GDP', 'CPI', 'Unemployment Rate', 'Federal Funds Rate',
       'Retail Sales']
macro.set_index('Date', inplace = True)
new_index = pd.date_range(start=start_date, end=end_date, freq='D')
macro.index = pd.to_datetime(macro.index)
macro = macro.ffill().reindex(new_index, method='ffill')
macro['Date'] = macro.index.astype(str)

In [27]:
temp = pd.merge(returns, macro, on='Date', how='left', suffixes=('_ret', '_MA'))
for variable in [factors, MA, rsi, macd, macd_s, B_m, B_u, B_l]:
    temp = pd.merge(temp, variable, on='Date', how='outer')

In [28]:
merged = temp.dropna().set_index('Date')
#merged.to_csv('Data/merged.csv')

In [29]:
len(merged)

1241